# week6_hw PCA and K-means

In [1]:
import jieba
import os
import pandas as pd
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import timedelta, date
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA as PCA
#from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [3]:
def is_chinese(uchar):         
    if u'\u4e00' <= uchar<=u'\u9fff':
        return True
    else:
        return False

In [4]:
# 專有名詞
jieba.load_userdict('ProperN.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/bf/kxrhqhkd3s1263kmtf84x8zw0000gn/T/jieba.cache
Loading model cost 0.713 seconds.
Prefix dict has been built succesfully.


In [5]:
folders = ["中電/", "金雨/", "大同/", "中興電/", "聲寶/", "東元/"]

In [6]:
imp_words = ["經營權",  "董事會",  "請辭", "接任", "出任", "兼任", "辭去", "入主", "推選", "另聘"]

In [7]:
def get_txts(folder):
    all_files = []
    for f in os.listdir("News/" + folder):
        if not f.startswith('.'):
            all_files.append(f)
    #os.listdir("News/" + folder)   # imagine you're one directory above test dir
    return all_files

In [8]:
# 讀取新聞檔案，並使用jieba切詞
def textMining(folder, file):
    with open("./News/" + folder + file, 'rt',encoding="utf-8", errors='ignore') as txt:
        data = txt.read()
    #text segmentation
    seg_list = jieba.cut(data, cut_all=False)
    seg_list = list(seg_list)
    filter_list = []
    wordDict = {}
    jump = False
    for word in seg_list:
        for s in word:
            if not is_chinese(s):
                jump = True
                break
        if not jump:
            filter_list.append(word)
        jump = False
    return filter_list

In [9]:
def analyze(folder, corpus, txts_in_folder):
    # tfidf
    vectorizer = TfidfVectorizer(max_df = 0.9, min_df = 0.2)
    tfidf = vectorizer.fit_transform(corpus)
    words = vectorizer.get_feature_names()
    print("tfidf.shape: ", tfidf.shape)
    for i in range(len(corpus)):
        print('----{0} NEWS----'.format(txts_in_folder[i]))
        for j in range(len(words)):
            if words[j] in imp_words:
                tfidf[i,j] = tfidf[i,j] * 2
            if tfidf[i,j] > 0.2:
                print(words[j], tfidf[i,j], i ,j)
    
    X = tfidf.toarray()
    sklearn_pca = PCA(n_components = 2)
    Y_sklearn = sklearn_pca.fit_transform(X)
    
    return Y_sklearn

In [10]:
def plot_cpa_scatter(Y_sklearn, txts_in_folder):
    pca_data = [
        go.Scatter(
            x = Y_sklearn[:,0],
            y = Y_sklearn[:,1],
            mode = "markers",
            hoverinfo = 'text',
            text = txts_in_folder
        )
    ]

    fig = go.Figure(data = pca_data)
    return py.iplot(fig, filename = 'PCA Scatter Chart')

In [11]:
def plot_kmeans_scatter(Y_sklearn, txts_in_folder):
    kmeans = KMeans(n_clusters = 3)
    X_clustered = kmeans.fit_predict(Y_sklearn)
    
    #Define our own color map
    LABEL_COLOR_MAP = {0:'red', 1: 'green', 2: 'blue'}
    label_color = [LABEL_COLOR_MAP[l] for l in X_clustered]
    
    pca_data = [
        go.Scatter(
            x = Y_sklearn[:,0],
            y = Y_sklearn[:,1],
            mode = "markers",
            hoverinfo = 'text',
            text = txts_in_folder,
            marker = dict(color = label_color)
        )
    ]

    fig = go.Figure(data = pca_data)
    return py.iplot(fig, filename = 'K-means Scatter Chart')

###  分析中電

In [12]:
# get corpus from folder 中電
txts_in_folder0 = get_txts(folders[0])
corpus0 = []

for file_name in txts_in_folder0:
    filter_list = textMining(folders[0], file_name)
    print(filter_list)
    join_list = " ".join(filter_list)
    corpus0.append(join_list)

Y_sklearn0 = analyze(folders[0], corpus0, txts_in_folder0)

['投保', '中心', '受理', '興航及', '中', '電求償', '登記', '投資人', '保護', '中心', '指出', '即日起', '受理', '興航', '含興', '航一可', '轉債', '受', '損害', '以及', '買進', '中電', '股票', '受', '損害', '之', '投資人', '求償', '登記', '符合', '資格', '投資人', '檢附', '文件', '及', '表格', '須在', '月', '日前', '寄回', '可電洽', '或', '查詢', '投保', '中心', '網站', '投保', '中心', '表示', '興航', '於', '年間', '涉有', '虛偽', '隱匿', '重大', '財務', '業務', '資訊', '等', '證券詐', '欺', '事件', '而', '有', '違反', '證券', '交易法', '之', '情事', '擬以', '相關', '事', '實及', '法條', '為', '基礎', '即日起', '受理', '善意', '投資人', '委任', '求償', '登記', '辦理', '民事', '求償', '事宜', '興航案', '受理', '條件', '一', '第一', '類投資人', '普通股', '買受', '人', '於', '年', '月', '日', '於', '市場', '買', '進', '復', '興', '航空', '普通股', '經', '扣除', '於', '同一', '日賣', '出', '所', '獲', '利益', '後', '仍受', '有', '損害', '者', '之', '善意', '買受', '人', '二', '第二', '類投資人', '公司債', '買受', '人', '於', '年', '月', '日', '於', '市場', '買', '進興', '航一可', '轉債', '經', '扣除', '於', '同一', '日賣', '出興', '航一可', '轉債', '所', '獲', '利益', '後', '仍受', '有', '損害', '者', '之', '善意', '買受', '人', '投保', '中心', '另', '指出', '中電', '對', '外', '公告', '之', '年

In [13]:
#plot_cpa_scatter(Y_sklearn0, txts_in_folder0)

In [14]:
plot_kmeans_scatter(Y_sklearn0, txts_in_folder0)

### 分析金雨

In [15]:
# get corpus from folder 金雨
txts_in_folder1 = get_txts(folders[1])
corpus1 = []

for file_name in txts_in_folder1:
    filter_list = textMining(folders[1], file_name)
    print(filter_list)
    join_list = " ".join(filter_list)
    corpus1.append(join_list)

Y_sklearn1 = analyze(folders[1], corpus1, txts_in_folder1)

['金雨', '世代交替', '卓政懋', '接', '董座', '年', '月', '日', '工商', '時報', '劉朱松', '台中', '報導', '國內', '自動', '販賣', '機龍頭', '廠', '商金雨', '企業', '啟動', '新一代', '接班', '金雨', '董事長', '兼', '總', '經理', '卓燦然', '昨', '日以', '興亞', '集團', '營建', '業務', '繁忙', '為', '由', '請辭', '金雨', '董事', '董事長', '及', '總', '經理', '職務', '經', '董事', '會', '決議', '改由', '其長', '子', '金雨', '董事長', '特助', '卓政懋', '接任', '董座', '次子', '金雨', '總', '經理', '特助', '卓政', '輿', '被', '聘為', '總座', '至於', '卓燦然', '的', '胞弟', '卓益', '豊', '原任', '金雨', '財務', '主管', '暨', '代理', '發言人', '的', '職務', '昨也', '改由', '金雨會計', '主管', '楊淑昭', '接任', '而卓益', '豊', '則', '出任', '興亞', '集團', '副', '董事長', '年次', '的', '卓政懋', '取得', '美國', '紐約', '大學', '營建', '管理', '碩士', '年次', '的', '卓政', '輿', '擁美國', '林肯', '大學行', '銷碩士', '學位', '均', '是', '卓燦然', '刻意', '栽培', '的', '下一代', '接班人', '卓政懋', '年底', '進入', '金雨', '先', '出任', '董事長', '特助', '後', '來還', '兼任', '金雨', '發言人', '去年', '月還', '出任', '專攻', '自動', '販賣', '機軟體', '支付', '系統', '的', '邦', '柏', '智慧', '科技', '董事長', '卓政', '輿', '年', '中', '先擔', '任金雨', '總', '經理', '特助', '協助', '公司', '持續', '承接', '自動', '販賣機'

/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [16]:
#plot_cpa_scatter(Y_sklearn1, txts_in_folder1)

In [17]:
plot_kmeans_scatter(Y_sklearn1, txts_in_folder1)

### 分析大同

In [18]:
# get corpus from folder 大同
txts_in_folder2 = get_txts(folders[2])
corpus2 = []

for file_name in txts_in_folder2:
    filter_list = textMining(folders[2], file_name)
    print(filter_list)
    join_list = " ".join(filter_list)
    corpus2.append(join_list)

Y_sklearn2 = analyze(folders[2], corpus2, txts_in_folder2)

['同', '昨', '現', '現', '理', '現', '現現', '現現', '現現', '現現', '現現', '現', '現現', '現現', '現現', '現現', '現', '來襲', '現現', '現現', '現現', '現現', '現', '現現', '現現', '現現', '現', '現現', '現現', '現現', '現', '現現', '現現', '陷', '現現', '現現', '現現', '現現', '現現', '現', '現現', '現現', '現現', '現現', '現現', '現', '派', '現現', '現現', '現', '現現', '現現', '現現', '現現', '現', '現現', '現現', '現現', '現現', '現現', '現現', '現現', '結', '的', '現現', '現', '現現', '現現', '現現', '現現', '現現', '現現', '現', '現現', '現現', '現現', '現現', '現現', '現', '現現', '現現', '現現', '現現', '億', '現現', '現現', '現現', '現現', '現現', '現現', '應有', '夠', '現現', '現', '現', '現現', '現現', '現', '現現', '現']
['大同', '年底', '所標', '得', '的', '桃園市', '設置', '太陽光', '電發', '電系', '統案', '已', '於', '年底', '順利', '完工', '結案', '該案', '是', '目前', '全', '台', '最大', '的', '掩埋', '場土', '地活', '用', '建置', '太陽能', '光電系', '統案', '總', '建置', '容量', '標', '百萬', '發電量', '可約', '達', '萬度電', '減', '公噸', '未來', '再', '擴充', '儲能', '系統', '等', '規劃', '可使', '整個', '桃園', '地區', '的', '太陽能', '發電', '併', '為', '智慧', '電網', '實現', '虛擬', '電廠', '概念', '大同', '表示', '垃圾', '掩埋', '場地質', '特殊', '更', '需要',

In [19]:
#plot_cpa_scatter(Y_sklearn2, txts_in_folder2)

In [20]:
plot_kmeans_scatter(Y_sklearn2, txts_in_folder2)

### 分析中興電 

In [21]:
# get corpus from folder 中興電
txts_in_folder3 = get_txts(folders[3])
corpus3 = []

for file_name in txts_in_folder3:
    filter_list = textMining(folders[3], file_name)
    print(filter_list)
    join_list = " ".join(filter_list)
    corpus3.append(join_list)

Y_sklearn3 = analyze(folders[3], corpus3, txts_in_folder3)

['鉅', '亨網', '離岸風電', '風場', '遴選', '送件將', '於', '明', '日', '截止', '遴選', '結果', '預計', '月', '下旬', '出爐', '在', '遴選', '結果', '出爐', '前', '今', '日買盤', '提前', '卡位', '風電', '族群', '包含', '上緯', '永冠', '等', '個', '股', '受惠', '遴選', '與', '政策', '題材', '帶動', '成', '今日', '盤', '面相', '對', '強勢', '的', '族群', '多達', '檔個', '股亮', '燈漲', '停', '上緯', '投控', '已', '成功', '架設', '並商轉', '百萬瓦', '示範風機', '今日', '開盤', '不久', '就強', '攻漲', '停價', '元鎖', '住', '創', '年', '月', '以來', '新高', '價', '永冠', '也', '不', '遑', '多', '讓', '同樣', '亮', '燈漲', '停', '站上', '波段', '新高', '元', '華城', '則是', '午盤', '後', '直奔', '漲', '停價', '元', '世紀', '鋼', '與', '台船', '大漲', '逾', '中興電', '則是', '上', '漲', '逾', '在', '離岸風電', '遴選即', '將出', '爐', '前夕', '不少', '國內外', '開發', '商與', '製造', '商積極', '簽署', '合作', '備忘錄', '盼能', '運用', '結盟', '的', '資金', '與', '技術', '優勢', '以及', '打造', '國產化供', '應鏈', '爭取', '通過', '遴選', '期在', '年前', '順利', '併', '網', '上緯', '先前', '就', '曾', '建置', '海洋', '風電', '第一', '階段', '示範風機', '近來', '也', '積極', '啟動', '相關', '佈', '局', '與', '國內外', '廠商', '簽署', '與', '永冠', '及', '先進', '複', '材', '簽署', '並與', '西門子', '歌

In [22]:
plot_kmeans_scatter(Y_sklearn3, txts_in_folder3)

### 分析聲寶

In [23]:
# get corpus from folder 聲寶
txts_in_folder4 = get_txts(folders[4])
corpus4 = []

for file_name in txts_in_folder4:
    filter_list = textMining(folders[4], file_name)
    print(filter_list)
    join_list = " ".join(filter_list)
    corpus4.append(join_list)

Y_sklearn4 = analyze(folders[4], corpus4, txts_in_folder4)

['聲寶', '集團', '昨日', '舉行', '台南', '廠', '新建', '工程', '動土', '第一期', '將斥資', '億元', '作為', '節能', '健康', '及', '環保', '的', '物聯網', '智能', '生活', '家電', '以及', '人工智慧', '相關', '產品', '生產據點', '該廠', '預計', '年', '完工', '挹注', '未來', '聲寶業績', '動能', '第二期', '規劃則', '增設', '研發', '大樓', '及', '物流', '專用', '倉庫', '等', '設施', '一', '二期', '合計', '投資', '超過', '億元', '聲寶總裁', '陳', '盛', '沺', '昨日', '主持', '台南', '廠', '新建', '工程', '動土', '儀式', '他', '說', '投資', '越南', '不如', '投資', '台南', '投資', '台灣', '也', '才能', '創造', '台灣', '更', '大', '的', '利基', '他', '強調', '包括', '他', '自己', '與', '公司', '總', '經理', '副', '總', '經理', '都', '是', '成大', '畢業', '深知', '台南', '擁有', '許多', '人才', '為', '因應', '未來', '公司', '成長', '需求', '台南', '廠將', '以', '少量多', '樣化', '的', '高效率', '生產', '結', '合成', '大等', '當地', '學校', '高端', '人才', '聲寶', '表示', '台南', '廠有', '二大', '功能', '一為', '促進', '南台', '灣經濟', '發展', '創造', '當地', '就業', '機會', '二是強', '化生', '產供', '應鏈', '及', '深耕', '在', '地', '客戶', '及', '擴大市', '占', '聲寶', '去年', '選定', '台南', '科技', '工業', '園區', '作為', '新建', '廠', '房基', '地', '占地', '坪', '為', '集團', '布局', '南台', '灣的', '重要',

In [24]:
plot_kmeans_scatter(Y_sklearn4, txts_in_folder4)

### 分析東元 

In [25]:
# get corpus from folder 東元
txts_in_folder5 = get_txts(folders[5])
corpus5 = []

for file_name in txts_in_folder5:
    filter_list = textMining(folders[5], file_name)
    print(filter_list)
    join_list = " ".join(filter_list)
    corpus5.append(join_list)

Y_sklearn5 = analyze(folders[5], corpus5, txts_in_folder5)

['東元', '去年', '營收', '突破', '億', '元年', '增', '東元', '去年', '合', '併', '營收', '突破', '億元', '年增', '達成', '現現', '現', '現現', '現現', '現現', '勢', '匯率', '影響', '應有', '更好', '表現', '現現', '現', '現現', '現', '現', '現現', '和', '太陽能', '發電', '都', '是', '營運中', '的', '重要', '項', '也', '是', '今年', '成', '的', '動能', '之', '東元', '公告', '結', '年', '營收', '現現', '現', '年', '現', '年', '營收', '現現', '年同', '期', '增加', '東元', '指出', '去年', '的確', '基期', '較', '主要', '是', '因', '為', '該', '有單', '筆處', '分收', '進帳', '所致', '東元', '是', '全球', '主', '的', '業', '達廠', '去年', '斥資', '逾', '億', '元', '的', '中壢', '座', '現', '現現', '現現', '成降', '整體', '效率', '提升', '三倍', '以上', '效率', '達產能', '將由年產', '萬台', '提', '到', '現現', '現', '年', '的', '績效', '將進', '步發揮', '現現', '現現', '現現', '現現', '現現', '現現', '現現', '現', '現現', '現', '現', '現', '萬', '興建太陽能', '發電系統', '發電量', '可達', '是', '邁向', '綠能', '發電', '的', '指標性', '投資案']
['東元', '首座', '太陽能', '廠', '今年', '併', '聯發電', '東元', '集團', '布局', '太陽能', '光電及', '風力', '發電', '等', '綠能產業', '進入', '收割期', '與茂迪', '合作', '在', '東元奈米', '觀音廠', '屋頂', '建置', '首座', '太陽能', '電廠', '近期', '完工', '併'

['聯合報', '記者', '高', '詩琴', '即時', '報導', '離岸風電', '大廠', '三菱重工維特斯', '繼與', '台塑', '上緯', '天力', '及', '中鋼', '簽署', '合作', '備忘錄', '後', '今天', '與', '東元電機', '董事長', '邱純枝', '簽署', '備忘錄', '三菱重工維特斯', '共同', '執行', '長', '今天', '表示', '台灣將', '採用', '大風機', '邱純枝', '表示', '最快', '年', '就', '會', '打造', '原型', '機', '東元電機', '董事長', '邱純枝', '表示', '離岸風電', '推國產化', '政策', '東元電機', '會當', '重要', '的', '零件', '供應商', '外商', '有嚴', '苛技術', '審查', '我們', '也', '一', '一通', '過', '證明', '東元', '在', '發', '電機', '項目', '上數', '一數', '二', '邱純枝', '指出', '年', '就', '會', '打造', '原型', '機', '並且', '接受', '第三方', '認證', '並且', '導入', '供', '應鏈', '比國家', '政策', '起碼', '提早', '一年', '也', '可以', '讓', '三菱重工維特斯', '早點', '達成', '在', '地化', '目標', '東元電機', '也', '期待', '三菱', '維', '特斯', '早點', '成立', '組裝廠', '成為', '領頭', '企業', '說', '目前', '三菱重工維特斯', '已經', '取得', '四個', '專案', '的', '優先議', '約權', '裝置', '容量', '約', '未來', '三菱重工維特斯', '也', '將爭', '取', '更', '多', '的', '業務', '以時間', '表來', '說', '約會', '在', '年', '至', '年', '之間', '開始', '安裝', '指出', '三菱重工維特斯', '會', '在', '台灣', '設立', '一個', '據點', '除了', '銷售', '以外', '也', '會', '有

['年', '月', '日', '工商', '時報', '文', '陳逸格', '馬達', '龍頭', '大廠', '東元電機', '身', '月', '日至', '日', '於', '台北', '國際', '自動化', '展華麗', '登場', '這次', '規模', '更勝', '以往', '共計', '個', '攤位', '規模', '與', '國內', '首屈一指', '商用', '通訊', '設備', '製造', '商', '關係企業', '東訊', '聯合', '策展', '主題', '為', '突顯', '其', '全球', '領先', '獨步國', '內業界', '之機', '電技術', '並', '符合', '東元', '節能', '減排', '智能', '自動', '之一', '貫目標', '本次', '產品', '陣容', '包含', '東元', '超高', '功率密度', '暨', '智能', '馬達', '全系列', '變頻', '器產品', '電控', '系統', '整合', '設備', '產品', '減速', '機產品', '太陽能', '器具', '產品', '東訊', '機電', '設備', '健康', '管理', '平台', '東訊', '攜', '帶式', '振動', '診斷', '儀', '等', '可望', '為', '集團營', '收成', '長', '增添', '動能', '同時', '為', '客戶', '創造', '更', '大', '效益', '其中', '的', '頭號', '亮點', '是', '東元', '集團', '歷經', '兩年', '研發', '的', '超高', '功率密度', '暨', '智能', '馬達', '成功', '突破', '全球', '框號', '馬達', '功率密度', '之', '上限', '馬達', '除了', '超高', '功率密度', '外', '也', '具備', '高效率', '輕量', '小型化', '及', '智能', '健康', '管理', '等', '特色', '東元', '指出', '特別', '適合', '搭配', '具緊', '湊型', '需求', '的', '風', '水力', '機產業', '設備', '應用', '於', '煉油廠', '及', '電廠

營收 0.2614295336010989 33 60
節能 0.27255468305770886 33 73
----20180801.txt NEWS----
功率密度 0.2580991103011658 34 10
台北 0.20407283814361585 34 16
整合 0.21951635809481576 34 50
智能 0.3035641222393337 34 53
服務 0.28677678922351757 34 55
自動化 0.22942143137881402 34 77
設備 0.4301651838352763 34 85
馬達 0.2907849995387541 34 103
----20180813.txt NEWS----
億元 0.86922900692042 35 5
合併營收 0.3192154911323619 35 19
增加 0.23974887495871938 35 24
----20180822.txt NEWS----
太陽能 0.7329361784245338 36 25
建置 0.3382782361959387 36 33
集團 0.4416548677996193 36 96


In [26]:
plot_kmeans_scatter(Y_sklearn5, txts_in_folder5)